In [7]:
def dbconnect():
    import pymysql
    import pandas as pd
    #DB연결
    con = pymysql.connect(host='ls-ca31b0251c0df0b74dc006a8cda1e13f6a53bf5b.cmjbqmtfh74s.ap-northeast-2.rds.amazonaws.com',
                          user='dbmasteruser',password='zaq1zaq!',db='dbmaster')
    cur = con.cursor()
    
    #실행할 쿼리 저장
    sql = "SELECT * FROM tb_data"
    
    cur.execute(sql)
    rows = cur.fetchall()
    
    #데이터 프레임으로 저장
    df = pd.DataFrame(rows)
    
    #불필요한 열 삭제
    del df[0],df[10]
    
    #열 이름 수정하기
    df.columns=['콘텐츠ID','콘텐츠분류','제목','위도','경도','태그','사진','소개','리뷰','지역분류','코사인','가중치','정규화']
    df['count']=0
    return(df)

In [33]:
def courserecommend(form):
    
    df=dbconnect()
    idlist0=[];idlist1=[];idlist2=[];idlist3=[]
    arr=[idlist1,idlist2,idlist0,idlist3]
#     region_type: 지역분류 
#     tour_tag: 관광지 해시태그 
#     food_tag: 음식 해시태그
#     lodge_tag: 숙박 해시태그
#     form={'region_type' : '서귀포시동부', 'tour_tag': '어트랙션,커플,역사유적',
#            'food_tag' : '갈치조림', 'lodge_tag': '주차장,바비큐'}
    region_type = form['region_type']; tour_tag=form['tour_tag'];food_tag=form['food_tag']; lodge_tag=form['lodge_tag']

    if region_type is not None :
        df=df.groupby(df['지역분류']).get_group(region_type)

    if tour_tag is not None:
        tour_tag=tour_tag.split(',')
        dft=df.groupby(df['콘텐츠분류']).get_group('관광지')

        for i in range(0,len(tour_tag)):
            for j in range(0,len(dft)):
                if dft.iloc[j,-1]>i:
                    dft.iloc[j,-1]=i
                if dft.iloc[j,5].find(tour_tag[i])!=-1 :
                    dft.iloc[j,-1]+=1



    if food_tag is not None:
        food_tag=food_tag.split(',')
        dff=df.groupby(df['콘텐츠분류']).get_group('음식점')

        for i in range(0,len(food_tag)):
            for j in range(0,len(dff)):
                if dff.iloc[j,-1]>i:
                    dff.iloc[j,-1]=i
                if dff.iloc[j,5].find(food_tag[i])!=-1 :
                    dff.iloc[j,-1]+=1


    if lodge_tag is not None:
        lodge_tag=lodge_tag.split(',')
        dfl=df.groupby(df['콘텐츠분류']).get_group('숙박')

        for i in range(0,len(lodge_tag)):
            for j in range(0,len(dfl)):
                if dfl.iloc[j,-1]>i:
                    dfl.iloc[j,-1]=i
                if dfl.iloc[j,5].find(lodge_tag[i])!=-1:
                    dfl.iloc[j,-1]+=1

                    
    # *****************************************************바뀐부분********************************************                
    import random

    #numt는 투어태그의 길이
    numt=len(tour_tag)

    #countt는 일치하는 태그가 많은것부터 비교하기 위한 변수로 겹치는 태그가 많은 행의 수

    countt=(dft['count']==numt).sum()


    
    if countt > 4:
        arrt=[i for i in range(0,countt)]
        
        for i in range(0,4):
            randomlistt=random.choice(arrt)
            arr[i].append(dft[dft['count']==numt].iloc[randomlistt,0])
            arrt.remove(randomlistt)
    else : 
        for i in range(0,4):
            arr[i].append(dft.sort_values(['count'],ascending=False).iloc[i,0])




    #-----------------------------------------------------------------------------------------------

    numf=len(food_tag)
    countf=(dff['count']==numf).sum()
    if countf > 4:
        arrf=[i for i in range(0,countf)]
        
        for i in range(0,4):
            randomlistf=random.choice(arrf)
            arr[i].append(dff[dff['count']==numf].iloc[randomlistf,0])
            arrf.remove(randomlistf)
    else : 
        for i in range(0,4):
            arr[i].append(dff.sort_values(['count'],ascending=False).iloc[i,0])




    #-----------------------------------------------------------------------------------------------               


    numl=len(lodge_tag)
    countl=(dfl['count']==numl).sum()
    if countl > 4:
        arrl=[i for i in range(0,countl)]
        
        for i in range(0,4):
            randomlistl=random.choice(arrl)
            arr[i].append(dfl[dfl['count']==numl].iloc[randomlistl,0])
            arrl.remove(randomlistl)
    else : 
        for i in range(0,4):
            arr[i].append(dfl.sort_values(['count'],ascending=False).iloc[i,0])

    return(arr)




In [37]:
form={'region_type' : '서귀포시동부', 'tour_tag': '어트랙션,커플,역사유적','food_tag' : '흑돼지', 'lodge_tag': '주차장,바비큐'}
courserecommend(form)

[['CNTS_200000000012075', 'CNTS_000000000001187', 'CNTS_000000000022573'],
 ['CONT_000000000500655', 'CNTS_000000000001177', 'CNTS_000000000019591'],
 ['CNTS_000000000020985', 'CNTS_000000000001125', 'CNTS_000000000018123'],
 ['CNTS_000000000022863', 'CONT_000000000501210', 'CNTS_000000000020390']]